This code iteratively solves for the correct ride and roll rate using the method described in section 16 of RCVD

In [ ]:
import numpy as np

# Variables

W_T = 569 + 180 # lbs, car + driver
W_bias = 0.507
W_f = W_bias*W_T # lbs, weight over front wheels
W_r = (1-W_bias)*W_T# lbs, weight over rear wheels
W_1 = W_f/2 #lbs, front left
W_2 = W_f/2 #lbs, front right
W_3 = W_r/2 #lbs, rear left
W_4 = W_r/2 #lbs, rear right

t_f = 51 #in, front track width
t_r = 51 #in, rear track width
l = 60 #in, wheelbase
a = (1-W_bias)*l
b = W_bias*l
h = 15 #in, CG height
H = 12.507 #in, CG to roll axis

z_rf = 2 #in, front roll axis height
z_rr = 3 #in, rear roll axis height

A_y = 1.2 #G, cornering acceleration

T_r = [743,737,715,683,652] #lb/in, Tire rates, camber from 0-4

K_RF = 167 # lb/in, ride rates
K_RR = 162

y_f_ideal = 0.6
y_r_ideal = 0.6

y_F = 1
y_R = 1

while y_R > y_r_ideal:
    K_rollF = 12*K_RF*t_f**2/2 # lb*ft/deg, roll rate
    K_rollR = 12*K_RR*t_r**2/2
    W_F = A_y*W_T/t_f*(H*K_rollF/(K_rollF+K_rollR)+b/l*z_rf)
    W_R = A_y*W_T/t_r*(H*K_rollR/(K_rollF+K_rollR)+a/l*z_rr)
    y_F = W_F/K_RF
    y_R = W_R/K_RR
    K_RF = W_F/y_f_ideal
    K_RR = W_R/y_r_ideal

freq_f = 1/(2*np.pi)*np.sqrt(K_RF*12*32.2/W_2)
freq_r = 1/(2*np.pi)*np.sqrt(K_RR*12*32.2/W_4)

print(f"The front ride rate is {K_RF} lb/in, the rear ride rate is {K_RR} lb/in")
print(f"the front wheel travel at {A_y} G's is {y_F} in, and the rear wheel travel is {y_R} in")
print(f"The front natural frequency is {freq_f} Hz, the rear natural frequency is {freq_r} Hz")




The front ride rate is 179.20436044453913 lb/in, the rear ride rate is 261.38387484957843 lb/in
the front wheel travel at 1.2 G's is 0.5999999999999998 in, and the rear wheel travel is 0.6 in
The front natural frequency is 3.03936659717943 Hz, the rear natural frequency is 3.7224486521744065 Hz
